In [1]:
import falcon
import psycopg2
import postgis
import json
import numpy as np



In [2]:
x_min = 2.35
x_max = 2.65
y_min = 48.85
y_max = 48.95


In [3]:
def limit_range(x_min, x_max, y_min, y_max, x_gap_max, y_gap_max):
    assert x_min <= x_max
    assert y_min <= y_max
    if x_max - x_min > x_gap_max:
        x_mean = (x_min + x_max)/2
        x_min = x_mean - (x_gap_max / 2.)
        x_max = x_mean + (x_gap_max / 2.)
    if y_max - y_min > y_gap_max:
        y_mean = (y_min + y_max)/2
        y_min = y_mean - (y_gap_max / 2.)
        y_max = y_mean + (y_gap_max / 2.)

    return x_min, x_max, y_min, y_max

In [ ]:

x_gap_max = 1.
y_gap_max = 0.5
x_min, x_max, y_min, y_max = limit_range(x_min, x_max, y_min, y_max, x_gap_max, y_gap_max)


In [4]:
connection = psycopg2.connect(dbname='solar', user='solar', password='baba')
cursor = connection.cursor()
postgis.register(cursor)


In [5]:

cursor.execute("""
    select commune, class_0_sum, class_1_sum, class_2_sum, class_3_sum, total, nom, wikipedia, surf_m2, geom
    from commune_merge
    where geom && st_makeenvelope(%s, %s, %s, %s, 4326);
    """, (x_min,y_min,x_max,y_max))

data = cursor.fetchall()

cursor.close()
connection.close()


In [11]:
data[0][6]

'12e Arrondissement'

In [20]:
features = []
for i, data_commune in enumerate(data):
    features.append({
            'type': 'Feature',
            'properties': {
                'commune': data_commune[0],
                'class_0_sum': data_commune[1],
                'class_1_sum': data_commune[2],
                'class_2_sum': data_commune[3],
                'class_3_sum': data_commune[4],
                'total': data_commune[5],
                'nom': data_commune[6],
                'wikipedia': data_commune[7],
                'surf_m2': float(data_commune[8]),
            },
            'geometry': data_commune[9].geojson
        })


In [ ]:

resp.status = falcon.HTTP_200
resp.set_header('Access-Control-Allow-Origin', '*')
resp.body = json.dumps({
       'type': 'FeatureCollection',
       'features': features,
    })


In [21]:
json.dumps(features[0]['properties']['surf_m2'])

'16319745.0'

In [15]:
features[0]

{'geometry': {'coordinates': ((((2.3991179630524995, 48.84809777992195),
     (2.415992546170099, 48.84661410673356),
     (2.4112265344302033, 48.83386688226201),
     (2.4221763689229214, 48.83581426867215),
     (2.4194236613977935, 48.842486688986654),
     (2.4221063901622886, 48.844498002237046),
     (2.427563116524144, 48.84150917375822),
     (2.437191888560036, 48.84089140314677),
     (2.4407661952440187, 48.84591647311163),
     (2.447732270946252, 48.84480151605461),
     (2.4617397011715276, 48.84269449867392),
     (2.467231551660993, 48.839093682202105),
     (2.469703445828304, 48.83555557941637),
     (2.4652295662188033, 48.8311513894441),
     (2.465761009540889, 48.82628238748528),
     (2.461307701351395, 48.81829837437693),
     (2.437349902218512, 48.81821911399533),
     (2.4302391977555358, 48.82340259731658),
     (2.419946391061623, 48.82415217603052),
     (2.4102443164985465, 48.825191596163435),
     (2.402487903009345, 48.82964687139496),
     (2.3902559